# Normal vs Leveraged Gains

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

## Gain Generator

This is a function that produces a series of random daily gains. The default parameters produces a cummulative gain of 10% (`target = 1.1`) over 200 days (`count = 200`), with a daily standard deviation 2% (`m = 0.02`). This closely resembles the SP-500. Users may also choose a uniform distribution by setting the `normal` keyword to be `False`.

In [ ]:
def gain_generator(target=1.1, count=200, m=0.02, verbose=0, normal=True):
    if normal:
        # Normally distributed numbers
        d = m * np.random.normal(size=count)
    else:
        # Uniform random numbers in between [-m, +m]
        d = 2.0 * m * np.random.random(count) - m
    # Force day 0 to be 0
    d[0]= 0.0
    # Find a gain that produces desired cumulative gain at the end of the period
    k = 0
    offset = 0;
    x = np.ones((count))
    while k < 20:
        x[1:] = 1.0 + d[1:] + offset
        ret = np.prod(x)
        delta = np.prod(x) - target
        if verbose:
            print('{:3d}  o = {:.6f} -> {:.6f}  {:+9.6f}'.format(k, offset, ret, delta))
        if abs(delta) < 1.0e-5:
            break
        offset -= 0.005 * delta
        k += 1
    return x - 1.0

In [ ]:
g = gain_generator()
np.std(g)

In [ ]:
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"

## Simulation

This simulation emulates a random daily return with the targeted compound gain and a 3x leveraged return

In [ ]:
# For a typical growth, low volatility, e.g., SPY, daily fluction can be higher
# m = 0.01; gain = 1.1; count = 200

# For high growth, high volatility, e.g., TQQQ, daily fluction can be higher
m = 0.015; gain = 1.09; count = 100

# Simulation
g = gain_generator(gain, count=count, m=m)
t = np.arange(count)
n = np.prod(1.0 + g)
l = np.prod(1.0 + 3.0 * g)
cn = 100 * (np.cumprod(1 + g) - 1)
cl = 100 * (np.cumprod(1 + 3.0 * g) - 1)
clr1 = 'tab:blue'
clr2 = 'tab:green'
delta = 0.5

# Raw Gain Values
plt.figure(dpi=120, figsize=(8, 3))
plt.plot(t, 100.0 * g, color=clr1, label='Normal')
plt.plot(t, 300.0 * g, color=clr2, label='Leveraged')
plt.grid(color='black', alpha=0.15)
plt.ylim((-11, 11))
plt.xlabel('Days')
plt.ylabel('Gain (%)')
plt.legend()
plt.title('Daily Gains', fontweight=800)

# Distribution of the Gains
plt.figure(dpi=120, figsize=(8, 3))
b = np.arange(-12 - 0.5 * delta, 12 + delta, delta)
n, _, patches = plt.hist(300 * g, bins=b, facecolor='green', edgecolor='k', alpha=0.5)
for b, p in zip(b, patches):
    if b < -0.5 * delta:
        p.set_facecolor('red')
    elif b < 0.5 * delta:
        p.set_facecolor('grey')
plt.xlim((-14, 14))
plt.grid(color='black', alpha=0.15)
plt.xlabel('Gain (%)')
plt.ylabel('Count')
plt.title('Distribution of Daily Leveraged Gains', fontweight=800)
plt.text(-13, 0.84 * max(n), 'Up Days = {}\nDown Days = {}'.format(
    np.sum(g >= 0), np.sum(g < 0)), linespacing=1.8)

# Cumulative Gain Over the Period
plt.figure(dpi=120, figsize=(8, 3))
plt.plot(t, cn, label='Normal', color=clr1)
plt.plot(t, cl, label='Leveraged', color=clr2)
plt.plot(t[-1] - 1, cn[-1], marker='o', color=clr1)
plt.text(t[-1] + 2, cn[-1], '{:.2f}%'.format(cn[-1]), color=clr1)
plt.plot(t[-1] - 1, cl[-1], marker='o', color=clr2)
plt.text(t[-1] + 2, cl[-1], '{:.2f}%'.format(cl[-1]), color=clr2)
plt.title('Cummulative Gain After {} Days'.format(count), fontweight=800)
plt.legend()
plt.xlabel('Days')
plt.ylabel('Gain (%)')
plt.grid(color='black', alpha=0.15)
plt.xlim((-0.04 * count, 1.14 * count))

print('')

In [ ]:
plt.close()

## Multiple Realizations

In [ ]:
# Number of realizations
r = 20

# Exercise
print('  Up Days      Normal    Leveraged         Extremes     Leveraged > Normal')
cc = 0
nn = 0
ll = 0
lab = {True: '+', False: ' '}
for k in range(r):
    g = gain_generator(gain, count, m=m)
    n = np.prod(1.0 + g)
    l = np.prod(1.0 + 3.0 * g)
    h = np.cumprod(1.0 + 3.0 * g)
    e = [100 * (x - 1) for x in (np.min(h), np.max(h))]
    c = np.sum(g >= 0)
    cc += c
    nn += n
    ll += l
    print('{:3d} ({:.1f}%)   {:5.2f}%     {:6.2f}%      ({:5.1f}%, {:5.1f}%)          {}'.format(
        c, 100 * c / count, 100 * (n - 1), 100 * (l - 1), e[0], e[1], lab[l > n]))
nn /= r
ll /= r

print('\nAverage from {} realizations:'.format(r))
print('Up days = {:.1f} ({:.1f}%)   normal = {:.2f}%   leveraged = {:.2f}%'.format(
    cc / r, 100 * cc / count / r, 100 * (nn - 1), 100 * (ll - 1)))